In [1]:
import pandas as pd
import numpy as np
import xgboost as xgb
import matplotlib.pylab as plt
from scipy.stats import pearsonr

from hyperopt import fmin, tpe, hp, partial


def find_time_ind(df):
    day_ind = [0]
    day_temp = df["TradingDay"][0]
    for i in range(1, len(df)):
        if df["TradingDay"][i] != day_temp:
            day_ind.append(i)
            day_temp = df["TradingDay"][i]
    return day_ind

In [2]:
# input cell
# train = pd.read_csv('0805rolling5.csv')
train = pd.read_csv('08053before_rolling.csv')
# train = pd.read_csv('08052acou_and_factors.csv')
# x_column = ["factor"+str(i)+'_'+str(j) for i in range(0,17) for j in range(5)]
x_column = ["factor"+str(i) for i in range(0,17)]
# for i in range(12):
#     for j in range(5):
#         x_column.append("10M" + str(i) +'_'+str(j))
for i in range(12):
    x_column.append("10M" + str(i))

train = train.sort_values(by = ["TradingDay", "SecuCode"], ascending = [True, True])
train.reset_index(drop=True, inplace=True)
X = train[x_column]
target = ['NextReturnCate']
y = train[target]
y += 1
yp = train["NextReturn"]
time_ind = find_time_ind(train)

In [3]:
# initializing
train_days = 775
delta_days = -776
X_train = X[time_ind[0]:time_ind[train_days]]
X_test = X[time_ind[train_days]:time_ind[train_days+delta_days]]
y_train = y[time_ind[0]:time_ind[train_days]]
y_test = y[time_ind[train_days]:time_ind[train_days+delta_days]]
yp_train = yp[time_ind[0]:time_ind[train_days]]
yp_test = yp[time_ind[train_days]:time_ind[train_days+delta_days]]
dtrain = xgb.DMatrix(data=X_train, label=y_train)
dtest = xgb.DMatrix(data=X_test)
ddev = xgb.DMatrix(data=X_test, label=y_test)

evallist = [(ddev, 'eval'), (dtrain, 'train')]


In [4]:
# 自定义hyperopt的参数空间
space = {"max_depth": hp.randint("max_depth", 25),
         "n_estimators": hp.randint("n_estimators", 30),
         'learning_rate': hp.uniform('learning_rate', 1e-3, 5e-1),
         "subsample": hp.randint("subsample", 5),
         "min_child_weight": hp.randint("min_child_weight", 6),
         "alpha": hp.uniform("alpha", 0, 1),
         "lambda": hp.uniform("lambda", 0, 1)
         }

def argsDict_tranform(argsDict, isPrint=False):
    argsDict["max_depth"] = argsDict["max_depth"] + 5    # from 5 to 30
    argsDict['n_estimators'] = argsDict['n_estimators'] + 10  # from 10 to 40
    argsDict["learning_rate"] = argsDict["learning_rate"] * 0.02 + 0.001   # from 0.001 to 0.01
    argsDict["subsample"] = argsDict["subsample"] * 0.1 + 0.5  # from 0.5 to 1
    argsDict["min_child_weight"] = argsDict["min_child_weight"] + 1 # from 1 to 6
    argsDict["alpha"] = argsDict["alpha"] * 20
    argsDict["lambda"] = argsDict["lambda"] * 50
    if isPrint:
        print(argsDict)
    else:
        pass

    return argsDict

In [5]:
from sklearn.metrics import mean_squared_error, zero_one_loss

def xgboost_factory(argsDict):
    argsDict = argsDict_tranform(argsDict)
    
    params = {'nthread': -1,  # 进程数
              'max_depth': argsDict['max_depth'],  # 最大深度
              'n_estimators': argsDict['n_estimators'],  # 树的数量
              'eta': argsDict['learning_rate'],  # 学习率
              'subsample': argsDict['subsample'],  # 采样数
              'min_child_weight': argsDict['min_child_weight'],  # 终点节点最小样本占比的和
              'objective': 'reg:squarederror',
              'silent': 0,  # 是否显示
              'gamma': 0,  # 是否后剪枝
              'colsample_bytree': 0.7,  # 样本列采样
              'alpha': argsDict['alpha'],  # L1 正则化
              'lambda': argsDict['lambda'],  # L2 正则化
              'scale_pos_weight': 0,  # 取值>0时,在数据不平衡时有助于收敛
              'seed': 100,  # 随机种子
              'tree_method':'exact'
              }

    xrf = xgb.train(params, dtrain, params['n_estimators'], evallist,early_stopping_rounds=100)

    return get_tranformer_score(xrf)


def get_tranformer_score(tranformer):
    
    xrf = tranformer
    pred = xrf.predict(ddev, ntree_limit=xrf.best_ntree_limit)
  
    return -pearsonr(np.array(yp_test), pred)[0]

In [6]:
# 开始使用hyperopt进行自动调参
algo = partial(tpe.suggest, n_startup_jobs=1)
best = fmin(xgboost_factory, space, algo=algo, max_evals=100, pass_expr_memo_ctrl=None)
store = best
print("the best parameter before transformation is:" + best)
print("the best parameter after transformation is:" + argsDict_tranform(best))


[0]	eval-rmse:0.956279	train-rmse:0.932308           

Multiple eval metrics have been passed: 'train-rmse' will be used for early stopping.


Will train until train-rmse hasn't improved in 100 rounds.

[1]	eval-rmse:0.956145	train-rmse:0.932158           

[2]	eval-rmse:0.955883	train-rmse:0.931955           

[3]	eval-rmse:0.955631	train-rmse:0.931761           

[4]	eval-rmse:0.955124	train-rmse:0.931316           

[5]	eval-rmse:0.95475	train-rmse:0.930898            

[6]	eval-rmse:0.954449	train-rmse:0.930655           

[7]	eval-rmse:0.954199	train-rmse:0.930452           

[8]	eval-rmse:0.954004	train-rmse:0.930189           

[9]	eval-rmse:0.953845	train-rmse:0.929862           

[10]	eval-rmse:0.953717	train-rmse:0.929499          

[11]	eval-rmse:0.953532	train-rmse:0.929097          

[12]	eval-rmse:0.953265	train-rmse:0.928752          

[13]	eval-rmse:0.953028	train-rmse:0.928432          

[14]	eval-rmse:0.952717	train-rmse:0.928113          

[15]	eval-rmse:0.952477	tra

[22]	eval-rmse:0.90745	train-rmse:0.879195                                         

[23]	eval-rmse:0.905828	train-rmse:0.877319                                        

[24]	eval-rmse:0.904193	train-rmse:0.875472                                        

[25]	eval-rmse:0.902475	train-rmse:0.873735                                        

[26]	eval-rmse:0.900919	train-rmse:0.871972                                        

[27]	eval-rmse:0.899502	train-rmse:0.870267                                        

[28]	eval-rmse:0.897922	train-rmse:0.868582                                        

[29]	eval-rmse:0.896487	train-rmse:0.866952                                        

[0]	eval-rmse:0.953651	train-rmse:0.929681                                         

Multiple eval metrics have been passed: 'train-rmse' will be used for early stopping.


Will train until train-rmse hasn't improved in 100 rounds.                        

[1]	eval-rmse:0.95095	train-rmse:0.926546                      

[14]	eval-rmse:0.925634	train-rmse:0.889295                                        

[15]	eval-rmse:0.92379	train-rmse:0.886836                                         

[16]	eval-rmse:0.921939	train-rmse:0.884399                                        

[17]	eval-rmse:0.920237	train-rmse:0.881871                                        

[18]	eval-rmse:0.918652	train-rmse:0.879373                                        

[19]	eval-rmse:0.916798	train-rmse:0.876919                                        

[20]	eval-rmse:0.915151	train-rmse:0.874572                                        

[21]	eval-rmse:0.913539	train-rmse:0.872188                                        

[22]	eval-rmse:0.912014	train-rmse:0.869761                                        

[23]	eval-rmse:0.910413	train-rmse:0.867382                                        

[24]	eval-rmse:0.908889	train-rmse:0.865071                                        

[25]	eval-rmse:0.907259	train-rmse:0.862838                      

[8]	eval-rmse:0.940482	train-rmse:0.911229                                         

[9]	eval-rmse:0.938684	train-rmse:0.909012                                         

[10]	eval-rmse:0.937166	train-rmse:0.9069                                          

[11]	eval-rmse:0.935419	train-rmse:0.904802                                        

[12]	eval-rmse:0.93396	train-rmse:0.902653                                         

[13]	eval-rmse:0.932458	train-rmse:0.900492                                        

[14]	eval-rmse:0.930935	train-rmse:0.898421                                        

[15]	eval-rmse:0.929358	train-rmse:0.896411                                        

[16]	eval-rmse:0.927872	train-rmse:0.894456                                        

[17]	eval-rmse:0.926322	train-rmse:0.892466                                        

[18]	eval-rmse:0.924872	train-rmse:0.890429                                        

[19]	eval-rmse:0.923149	train-rmse:0.888404                      

[30]	eval-rmse:0.91086	train-rmse:0.876513                                          

[31]	eval-rmse:0.909636	train-rmse:0.875004                                         

[32]	eval-rmse:0.90856	train-rmse:0.87353                                           

[33]	eval-rmse:0.907414	train-rmse:0.872087                                         

[0]	eval-rmse:0.954117	train-rmse:0.930414                                          

Multiple eval metrics have been passed: 'train-rmse' will be used for early stopping.


Will train until train-rmse hasn't improved in 100 rounds.                          

[1]	eval-rmse:0.952296	train-rmse:0.927796                                          

[2]	eval-rmse:0.950253	train-rmse:0.925694                                          

[3]	eval-rmse:0.948292	train-rmse:0.923453                                          

[4]	eval-rmse:0.946414	train-rmse:0.921269                                          

[5]	eval-rmse:0.944635	train-rmse:0.919208          

...

[31]	eval-rmse:0.91764	train-rmse:0.890668                                          

[32]	eval-rmse:0.916572	train-rmse:0.88956                                          

[33]	eval-rmse:0.915499	train-rmse:0.888468                                         

[34]	eval-rmse:0.914612	train-rmse:0.887402                                         

[35]	eval-rmse:0.913666	train-rmse:0.88633                                          

[36]	eval-rmse:0.912712	train-rmse:0.88524                                          

[37]	eval-rmse:0.911775	train-rmse:0.884216                                         

[38]	eval-rmse:0.910707	train-rmse:0.883205                                         

[0]	eval-rmse:0.955914	train-rmse:0.931959                                          

Multiple eval metrics have been passed: 'train-rmse' will be used for early stopping.


Will train until train-rmse hasn't improved in 100 rounds.                          

[1]	eval-rmse:0.954915	train-rmse:0.931001          

[17]	eval-rmse:0.938689	train-rmse:0.90334                                          

[18]	eval-rmse:0.937819	train-rmse:0.901749                                         

[19]	eval-rmse:0.936815	train-rmse:0.900207                                         

[0]	eval-rmse:0.954711	train-rmse:0.930874                                          

Multiple eval metrics have been passed: 'train-rmse' will be used for early stopping.


Will train until train-rmse hasn't improved in 100 rounds.                          

[1]	eval-rmse:0.953411	train-rmse:0.928882                                          

[2]	eval-rmse:0.951773	train-rmse:0.927095                                          

[3]	eval-rmse:0.950313	train-rmse:0.925593                                          

[4]	eval-rmse:0.948895	train-rmse:0.923908                                          

[5]	eval-rmse:0.947452	train-rmse:0.922094                                          

[6]	eval-rmse:0.946019	train-rmse:0.920478          

[7]	eval-rmse:0.943533	train-rmse:0.917033                                          

[8]	eval-rmse:0.942054	train-rmse:0.915268                                          

[9]	eval-rmse:0.940511	train-rmse:0.913507                                          

[10]	eval-rmse:0.939107	train-rmse:0.911863                                         

[11]	eval-rmse:0.937559	train-rmse:0.910164                                         

[12]	eval-rmse:0.936152	train-rmse:0.908496                                         

[13]	eval-rmse:0.93457	train-rmse:0.906837                                          

[14]	eval-rmse:0.933142	train-rmse:0.905154                                         

[15]	eval-rmse:0.931717	train-rmse:0.903478                                         

[16]	eval-rmse:0.930327	train-rmse:0.901864                                         

[17]	eval-rmse:0.928905	train-rmse:0.900232                                         

[18]	eval-rmse:0.927604	train-rmse:0.898553           

[26]	eval-rmse:0.925951	train-rmse:0.900084                                         

[0]	eval-rmse:0.955869	train-rmse:0.931897                                          

Multiple eval metrics have been passed: 'train-rmse' will be used for early stopping.


Will train until train-rmse hasn't improved in 100 rounds.                         

[1]	eval-rmse:0.954793	train-rmse:0.930813                                         

[2]	eval-rmse:0.954159	train-rmse:0.929913                                         

[3]	eval-rmse:0.953488	train-rmse:0.928748                                         

[4]	eval-rmse:0.952799	train-rmse:0.927763                                         

[5]	eval-rmse:0.952143	train-rmse:0.926911                                         

[6]	eval-rmse:0.951466	train-rmse:0.926124                                         

[7]	eval-rmse:0.950763	train-rmse:0.925314                                         

[8]	eval-rmse:0.950101	train-rmse:0.924448                  

TypeError: must be str, not dict

In [7]:
best

{'alpha': 0.024323280644326623,
 'lambda': 0.0686095965507202,
 'learning_rate': 0.12013231877185954,
 'max_depth': 6,
 'min_child_weight': 2,
 'n_estimators': 25,
 'subsample': 2}

In [8]:
argsDict_tranform(best)

{'alpha': 0.48646561288653245,
 'lambda': 3.43047982753601,
 'learning_rate': 0.003402646375437191,
 'max_depth': 11,
 'min_child_weight': 3,
 'n_estimators': 35,
 'subsample': 0.7}

In [47]:
best

{'learning_rate': 0.415602004895926,
 'max_depth': 2,
 'min_child_weight': 5,
 'n_estimators': 14,
 'subsample': 2}

In [49]:
 xgboost_factory(argsDict_tranform(best))

[0]	eval-rmse:0.887142	train-rmse:0.91346
Multiple eval metrics have been passed: 'train-rmse' will be used for early stopping.

Will train until train-rmse hasn't improved in 100 rounds.
[1]	eval-rmse:0.876897	train-rmse:0.894919
[2]	eval-rmse:0.868053	train-rmse:0.878668
[3]	eval-rmse:0.859192	train-rmse:0.86251
[4]	eval-rmse:0.851483	train-rmse:0.848265
[5]	eval-rmse:0.844992	train-rmse:0.833192
[6]	eval-rmse:0.838924	train-rmse:0.820392
[7]	eval-rmse:0.833219	train-rmse:0.807377
[8]	eval-rmse:0.828818	train-rmse:0.794724
[9]	eval-rmse:0.824214	train-rmse:0.783454
[10]	eval-rmse:0.820525	train-rmse:0.772966
[11]	eval-rmse:0.817106	train-rmse:0.762479
[12]	eval-rmse:0.814711	train-rmse:0.752645
[13]	eval-rmse:0.812286	train-rmse:0.743479
[14]	eval-rmse:0.810634	train-rmse:0.734102
[15]	eval-rmse:0.808537	train-rmse:0.72533
[16]	eval-rmse:0.807054	train-rmse:0.716354
[17]	eval-rmse:0.805628	train-rmse:0.707871
[18]	eval-rmse:0.804132	train-rmse:0.699963
[19]	eval-rmse:0.803122	train-r

-0.06243747878596423

In [31]:
bst=xgb.train(params,dtrain=dtrain,num_boost_round=1,early_stopping_rounds=None,evals=[(dtrain,'train'),(ddev,'test')],verbose_eval=3)


[0]	train-rmse:0.926187	test-rmse:0.896756


In [36]:
xrf = xgb.train(params, dtrain,num_boost_round=1,early_stopping_rounds=None)

In [ ]:
# first time training

bst=xgb.train(params=xgb_param,dtrain=dtrain,num_boost_round=1,early_stopping_rounds=None,evals=[(dtrain,'train'),(ddev,'test')],verbose_eval=3)
pred = bst.predict(dtest)
predtrain = bst.predict(dtrain)
print(pearsonr(np.array(yp_train), predtrain))
print(pearsonr(np.array(yp_test), pred))


# first time training - continued
for kk in range(29):
    bst=xgb.train(params=xgb_param,dtrain=dtrain,num_boost_round=1,early_stopping_rounds=None,evals=[(dtrain,'train'),(ddev,'test')],verbose_eval=3, xgb_model = bst)
    pred = bst.predict(dtest)
    predtrain = bst.predict(dtrain)
print(pearsonr(np.array(yp_train), predtrain))
# print(pearsonr(np.array(y_train.T)[0], predtrain))
print(pearsonr(np.array(yp_test), pred))
# print(pearsonr(np.array(y_test.T)[0], pred))

# concatenate predictions

g =  train[["TradingDay", "SecuCode"]][time_ind[train_days]:time_ind[train_days+delta_days]]
g["NextCatPred"] = pred
k = g.pivot_table(columns='SecuCode', index='TradingDay', values='NextCatPred').reset_index().sort_values("TradingDay")


In [ ]:
# second traininng and so on

# change dataset
drift = 1
while drift * delta_days + train_days + delta_days < len(time_ind):
    X_train = X[time_ind[drift * delta_days]:time_ind[train_days + drift * delta_days]]
    X_test = X[time_ind[train_days + drift * delta_days]:time_ind[train_days+delta_days + drift * delta_days]]
    y_train = y[time_ind[drift * delta_days]:time_ind[train_days + drift * delta_days]]
    y_test = y[time_ind[train_days + drift * delta_days]:time_ind[train_days+delta_days + drift * delta_days]]
    yp_train = yp[time_ind[drift * delta_days]:time_ind[train_days + drift * delta_days]]
    yp_test = yp[time_ind[train_days + drift * delta_days]:time_ind[train_days+delta_days + drift * delta_days]]
    dtrain = xgb.DMatrix(data=X_train, label=y_train)
    dtest = xgb.DMatrix(data=X_test)
    ddev = xgb.DMatrix(data=X_test, label=y_test)
    bst=xgb.train(params=xgb_param,dtrain=dtrain,num_boost_round=1,early_stopping_rounds=None,evals=[(dtrain,'train'),(ddev,'test')],verbose_eval=3)

    for kk in range(29):
        bst=xgb.train(params=xgb_param,dtrain=dtrain,num_boost_round=1,early_stopping_rounds=None,evals=[(dtrain,'train'),(ddev,'test')],verbose_eval=3, xgb_model = bst)
#         bst=xgb.train(params=xgb_param,dtrain=dtrain,num_boost_round=1,early_stopping_rounds=None,evals=[(dtrain,'train'),(ddev,'test')],verbose_eval=3)
        pred = bst.predict(dtest)
        predtrain = bst.predict(dtrain)
    print(pearsonr(np.array(yp_train), predtrain))
    # print(pearsonr(np.array(y_train.T)[0], predtrain))
    print(pearsonr(np.array(yp_test), pred))
    # print(pearsonr(np.array(y_test.T)[0], pred))
    gt =  train[["TradingDay", "SecuCode"]][time_ind[train_days + drift * delta_days]:time_ind[train_days+delta_days + drift * delta_days]]
    gt["NextCatPred"] = pred
    kt = gt.pivot_table(columns='SecuCode', index='TradingDay', values='NextCatPred').reset_index().sort_values("TradingDay")
    k = pd.concat([k, kt],ignore_index=True)
    drift += 1

# the last dataset
if drift * delta_days + train_days + delta_days > len(time_ind):
    X_train = X[time_ind[drift * delta_days]:time_ind[train_days + drift * delta_days]]
    X_test = X[time_ind[train_days + drift * delta_days]:]
    y_train = y[time_ind[drift * delta_days]:time_ind[train_days + drift * delta_days]]
    y_test = y[time_ind[train_days + drift * delta_days]:]
    yp_train = yp[time_ind[drift * delta_days]:time_ind[train_days + drift * delta_days]]
    yp_test = yp[time_ind[train_days + drift * delta_days]:]
    dtrain = xgb.DMatrix(data=X_train, label=y_train)
    dtest = xgb.DMatrix(data=X_test)
    ddev = xgb.DMatrix(data=X_test, label=y_test)
    bst=xgb.train(params=xgb_param,dtrain=dtrain,num_boost_round=1,early_stopping_rounds=None,evals=[(dtrain,'train'),(ddev,'test')],verbose_eval=3)

    for kk in range(29):
        bst=xgb.train(params=xgb_param,dtrain=dtrain,num_boost_round=1,early_stopping_rounds=None,evals=[(dtrain,'train'),(ddev,'test')],verbose_eval=3, xgb_model = bst)
        pred = bst.predict(dtest)
        predtrain = bst.predict(dtrain)
    print(pearsonr(np.array(yp_train), predtrain))
    # print(pearsonr(np.array(y_train.T)[0], predtrain))
    print(pearsonr(np.array(yp_test), pred))
    # print(pearsonr(np.array(y_test.T)[0], pred))
    gt =  train[["TradingDay", "SecuCode"]][time_ind[train_days + drift * delta_days]:]
    gt["NextCatPred"] = pred
    kt = gt.pivot_table(columns='SecuCode', index='TradingDay', values='NextCatPred').reset_index().sort_values("TradingDay")
    k = pd.concat([k, kt],ignore_index=True)

k.set_index("TradingDay").to_csv("0805back_input_30_30.csv")